# STRING - Classic preprocessing

Este notebook es una primera aproximación donde se preprocesarán las interacciones físicas directas registradas en el dataset STRING y se codificarán numéricamente para en el siguiente notebook hacer el entrenamiento del modelo.

Para ello, se ha descargado el dataset con interacciones físicas filtrado por la especie homo sapiens.

## 1. Exploración de datos

In [ ]:
import pandas as pd

db_string_path = 'datasets/9606.protein.physical.links.full.v12.0.txt'
df_string = pd.read_csv(db_string_path, sep=' ', low_memory=False)

df_string.head()

In [ ]:
df_string.shape

In [ ]:
df_string.info()

In [ ]:
df_string.isna().sum().sum()

## 2. Limpieza y preparación de datos
El dataset de STRING que hemos descargado indica en la propia web: "incl. distinction: direct vs. interologs". Esto significa que se indica qué interacciones son directas o por interología. Para nuestro proyecto buscamos aquellas directas comprobadas en humanos mediante técnicas experimentales. Estas son las que tienen experimentos registrados.

In [ ]:
df_direct = df_string[df_string['experiments'] > 0]
print(f"Total de interacciones con evidencia experimental directa: {len(df_direct)}")
df_direct.head()

Ahora que tenemos aquellas proteínas con interacción física directa, obtenemos el listado de proteínas únicas y nos quedamos con los pares de proteínas conocidos (estos serán nuestros casos positivos) para luego generar pares que no estén registrados (estos serán los casos negativos).

In [ ]:
# Obtenemos todas las proteínas únicas
all_proteins = pd.unique(df_string[['protein1', 'protein2']].values.ravel())
print(f"Número de proteínas únicas: {len(all_proteins)}")

In [ ]:
positive_pairs = set(
    tuple(sorted([row['protein1'], row['protein2']]))
    for _, row in df_direct.iterrows()
)

In [ ]:
import random

negative_pairs = set()

while len(negative_pairs) < len(df_direct):
    p1, p2 = random.sample(list(all_proteins), 2)
    pair = tuple(sorted([p1, p2]))
    if pair not in positive_pairs:
        negative_pairs.add(pair)

Acabamos de generar el set de casos negativos para nuestro modelo. Sin embargo, hay que aclarar que aquí nos encontramos ante uno de los mayores desafíos conceptuales de las PPI: los casos negativos no implica que esas dos proteínas se haya demostrado que no interactúen entre ellas, implica que su interacción no está recogida dentro del listado de interacciones conocidas. Esto nos dice que un par del set de negativos podría en realidad dar lugar a una interacción positiva si se hiciese una prueba experimental. Entonces, **asumimos que los pares "negativos" no es que no interactúen entre ellas, sino que desconocemos su interacción**.

In [ ]:
df_negative = pd.DataFrame(list(negative_pairs), columns=['protein1', 'protein2'])
df_negative['label'] = 0

df_positive = df_direct[['protein1', 'protein2']].copy()
df_positive['label'] = 1

df_string_balanced = pd.concat([df_positive, df_negative], ignore_index=True)

In [ ]:
import importlib
import src.fasta_parser
importlib.reload(src.fasta_parser)

from src.fasta_parser import FastaParser

fasta_parser = FastaParser("datasets/Homo_sapiens.GRCh38.pep.all.fa")
protein_seqs = fasta_parser.to_dict()

print(f"Total de proteínas indexadas: {len(protein_seqs)}")

In [ ]:
for i, (ensembl_id, seq) in enumerate(protein_seqs.items()):
    print(f"{ensembl_id}: {seq[:50]}...")
    if i >= 4:
        break

In [ ]:
df_string_balanced["sequence1"] = df_string_balanced["protein1"].apply(
    lambda p: protein_seqs.get(fasta_parser.extract_ensembl_id(p), None)
)
df_string_balanced["sequence2"] = df_string_balanced["protein2"].apply(
    lambda p: protein_seqs.get(fasta_parser.extract_ensembl_id(p), None)
)

In [ ]:
df_string_balanced.shape

In [ ]:
df_string_balanced.isna().sum().sum()

Observamos que hay 66626 NaNs en nuestro DF tras añadir las secuencias de aminoácidos. Investigamos qué ocurre.

In [ ]:
from src.utils import analyze_missing_proteins

analyze_missing_proteins(df_string_balanced, protein_seqs)

Parece que el problema es que 366 proteínas del dataset de STRING no se encuentran en el archivo FASTA de Ensembl. Esto puede deberse a distintos factores, como que las proteínas hayan sido renombradas, retiradas o aún no incluídas en el fichero de Ensembl. Teniendo en cuenta que tenemos un dataset con 1.894.316 entradas, eliminar 66.626 no se considera un problema.

In [ ]:
df_string_balanced.dropna(inplace=True)

## Sequence encoding

Antes de codificar y preparar las secuencias proteicas como entrada al modelo, tenemos que determinar una longitud máxima (`max_length`) para todas ellas. Los modelos de deep learning, especialmente aquellos que trabajan con batches, requieren entradas de tamaño fijo.

Para definir un `max_length` adecuado, analizaremos la distribución de longitudes de todas las secuencias de proteínas involucradas en los pares del dataset. Esto nos permitirá encontrar un equilibrio entre:

- Maximizar la cobertura (evitar truncar demasiadas secuencias)
- Minimizar el uso de memoria y el tiempo de entrenamiento

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Crear copia y calcular longitudes
df_lengths = pd.DataFrame()
df_lengths["len_seq1"] = df_string_balanced["sequence1"].str.len()
df_lengths["len_seq2"] = df_string_balanced["sequence2"].str.len()

lengths = pd.concat([df_lengths["len_seq1"], df_lengths["len_seq2"]])

print(lengths.describe())
print("Cuantiles:")
print(lengths.quantile([0.5, 0.75, 0.90, 0.95, 0.99]))

plt.hist(lengths, bins=100, edgecolor="black")
plt.title("Distribución de longitudes de secuencias")
plt.xlabel("Longitud")
plt.ylabel("Frecuencia")
plt.xlim(0, 2000)
plt.grid(True)
plt.show()


El histograma muestra que la gran mayoría de las secuencias tienen una longitud inferior a 1000 aminoácidos. En concreto, los percentiles calculados reflejan que:

- El 90 % de las secuencias tienen ≤ 1174 aminoácidos
- El 95 % tienen ≤ 1622
- El 99 % tienen ≤ 2839

Basándonos en esta distribución, se ha decidido establecer un `max_length` de **1024**. Este valor permite cubrir hasta casi el 90 % de las secuencias sin truncamiento, al tiempo que mantiene un consumo de memoria razonable compatible con la GPU disponible. Las secuencias más largas serán truncadas para ajustarse a esta longitud, mientras que las más cortas serán completadas mediante padding.

In [ ]:
from src.encoders.numeric_sequence_encoder import NumericSequenceEncoder
from src.sequence_preprocessor import SequencePreprocessor

encoder = NumericSequenceEncoder()
preprocessor = SequencePreprocessor(encoder, max_length=1024)

df_encoded = preprocessor.process_dataframe(df_string_balanced)
df_encoded.to_parquet("processed_data/classic_encoded_1024.parquet")

### Conclusión del preprocesamiento

En este primer notebook se ha llevado a cabo el preprocesamiento completo del dataset de interacciones proteína-proteína (PPI) con evidencia experimental directa, extraído de la base de datos STRING.

El proceso incluyó la asociación de secuencias de aminoácidos mediante identificadores Ensembl, la codificación numérica de las secuencias con una longitud máxima de 1024 residuos, y el etiquetado binario de las interacciones (positivas y negativas). Para ello se implementaron clases reutilizables siguiendo principios de diseño limpio y modular, como `SequencePreprocessor` y `SequenceEncoder`.

El resultado final es un archivo en formato `.parquet` que contiene los datos preprocesados y balanceados, listos para ser utilizados en la fase de entrenamiento del modelo de deep learning. Este enfoque desacopla el procesamiento de datos del entrenamiento, facilitando futuras pruebas con distintos modelos y codificadores sin necesidad de repetir esta etapa.
